# Please go to https://ccv.jupyter.brown.edu

## What we learned so far...
- The main goal of NLP in machine learning
- Text normalization
- The differences between stemming and lemmatization
- Bag of Words

# 5. Intro to Natural Language Processing (NLP)
#### By the end of this day, you'll be able to 
- explain the differences between bag of words and n-grams
- apply the TFIDF transformation
- explain the difference between topic modeling and LDA
- run through a simple NLP pipeline, from cleaning and vectorizing text to mining topics

## 5.1 N-grams
- an ngram is a contiguous sequence of n items from a given sample of text
- the items can be syllables, letters, words, or base pairs (according to the application)

- we can generalize bag of words to phrases of *n* words
- bag of words is a unigram representation of text
- we can have unigrams, bigrams, 3-grams, 4-grams, etc.

#### A document:
- 'It was the best of times'

#### Bigrams:   ['it was', 'was the', 'the best', 'best of', 'of times']
#### Trigrams:   ['it was the', 'was the best', 'the best of', 'best of times']
#### 4-grams: ['it was the best', 'was the best of', 'the best of times']

### 5.1.2 Bag of words using n-grams

- changing the unit of analysis from words to n-grams can help to encode some contextual information in NLP applications
- `CountVectorizer()` has a built-in `ngrams_range` option for computing a count matrix with any number of n-grams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['It was the best of times,',
          'it was the worst of times,',
          'it was the Age of Wisdom,',
          'it was the Age of Foolishness,']

vectorizer = CountVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(corpus)
count_matrix = X.toarray()
bigrams = vectorizer.get_feature_names()

print(count_matrix)
print(bigrams)

### Construct a count matrix with a vocabulary made up of both bigrams and trigrams. Print the vocabulary and the count array.

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2,3))
X = vectorizer.fit_transform(corpus)
count_matrix = X.toarray()
bigrams = vectorizer.get_feature_names()

print(count_matrix)
print(bigrams)

# Exercise 1
### Construct a count matrix with a vocabulary made up of trigrams. Print the vocabulary and the count array.

In [ ]:
# Solution


### 5.1.3 Bag of words vs. N-grams

- bag of words is simple, but it is computationally inefficient
- n-grams can create an even larger count matrix
- corpus of 1 billion ($ 10^9 $) words contains roughly $ 10^5 $ 1-grams, $ 3 \times 10^5 $ 2-grams, over $ 10^6 $ 3-grams
- n > 3 is rarely used
- One counter-example: a large n can reveal plagarism 

## 5.2 Towards the TFIDF transformation:

### 5.2.1 Another look at the count matrix

|c|term_1|term_2|...|term_j|...|term_m|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|__Doc_1__|c_11|c_12|...|c_1j|...|c_1m|
|__Doc_2__|c_21|c_22|...|c_2j|...|c_2m|
|__...__|...|...|...|...|...|...|
|__Doc_i__|c_i1|c_i2|...|c_ij|...|c_im|
|__...__|...|...|...|...|...|...|
|__Doc_n__|c_n1|c_n2|...|c_nj|...|c_nm|

- c is our count matrix
- c_ij = number of times term_j appears in document_i

### 5.2.2 The term count vector

|-|term_1|term_2|...|term_j|...|term_m|__<font color='red'>T</font>__|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|__Doc_1__|c_11|c_12|...|c_1j|...|c_1m|__<font color='red'>sum_j(c_1j)</font>__|
|__Doc_2__|c_21|c_22|...|c_2j|...|c_2m|__<font color='red'>sum_j(c_2j)</font>__|
|__...__|...|...|...|...|...|...|<font color='red'>...</font>|
|__Doc_i__|c_i1|c_i2|...|c_ij|...|c_im|__<font color='red'>sum_j(c_ij)</font>__|
|__...__|...|...|...|...|...|...|__<font color='red'>...</font>|
|__Doc_n__|c_n1|c_n2|...|c_nj|...|c_nm|__<font color='red'>sum_j(c_nj)</font>__|

- T is the term count vector
- T_i is the number of terms in document_i

### 5.2.3 The document count vector

|-|term_1|term_2|...|term_j|...|term_m|__<font color='red'>T</font>__|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|__Doc_1__|c_11|c_12|...|c_1j|...|c_1m|__<font color='red'>sum_j(c_1j)</font>__|
|__Doc_2__|c_21|c_22|...|c_2j|...|c_2m|__<font color='red'>sum_j(c_2j)</font>__|
|__...__|...|...|...|...|...|...|<font color='red'>...</font>|
|__Doc_i__|c_i1|c_i2|...|c_ij|...|c_im|__<font color='red'>sum_j(c_ij)</font>__|
|__...__|...|...|...|...|...|...|__<font color='red'>...</font>|
|__Doc_n__|c_n1|c_n2|...|c_nj|...|c_nm|__<font color='red'>sum_j(c_nj)</font>__|
|__<font color='blue'>D</font>__|__<font color='blue'>sum_i(c_i1 > 0)</font>__|__<font color='blue'>sum_i(c_i2 > 0)</font>__|<font color='blue'>...</font>|__<font color='blue'>sum_i(c_ij > 0)</font>__|<font color='blue'>...</font>|__<font color='blue'>sum_i(c_im > 0)</font>__||

- D is the document count vector
- D_j is the number of documents that contain term_j at least once.

# Exercise 2
### Calculate (by hand) the term count vector and document count vector of the following count matrix:

|-|term_1|term_2|term_3|term_4|term_5|term_6|term_7|term_8|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|__Doc1__|2|3|1|0|3|3|1|2|
|__Doc2__|1|3|1|0|0|0|3|2|
|__Doc3__|1|0|1|2|0|1|0|1|
|__Doc4__|0|2|1|3|3|0|0|3|
|__Doc5__|2|2|2|0|1|0|3|2|
|__Doc6__|1|0|3|3|1|2|3|1|
|__Doc7__|2|2|0|2|0|2|3|2|


### Solution




## 5.3 The TFIDF transformation

#### We need to normalize or de-bias the count matrix!
- Some documents are shorter, others are longer => there is a bias towards longer documents
- Some terms appear in most of the documents => there is bias towards frequent terms

### 5.3.1 TFIDF - Term Frequency times Inverse Document Frequency

1. Compute the weights, W_ij = tf * idf
    - tf = C_ij/T_i (frequency of word in a document)
    - idf = ln(n/D_j)
    - df = D_j/n (rank the word for how many documents it shows up in)
    - n = number of documents
#### <center>W_ij = c_ij / T_i * ln(n / D_j)</center>

2. Normalize the weights so the weights are between 0 and 1 for each document
#### <center>TFIDF_ij = W_ij / sqrt( sum( W_ij^2 ) )</center>

### That was the textbook definition of TFIDF. The `sklearn` package defines TFIDF in a slightly more complicated way...

#### <center>W_ij = (c_ij / T_i) * (ln[(n + 1)/(D_j + 1)]+1)</center>
- idf(sklearn) = ln[(n+1)/(D_j+1)]+1
    - adding 1 to numerator and denominator prevents zero divisions (as if extra document was seen containing every term in the corpus exactly once)
    - adding 1 to idf prevents terms with zero idf from being ignored (terms that occur in all documents have zero idf)

### 5.3.2 Apply TFIDF to a count matrix


|__W__ |term_1|term_2|...|term_j|...|term_m|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|__Doc_1__|( c_11/T_1 )*( ln[(n+1)/(D_1+1)]+1 )|( c_12/T_1 )*( ln[(n+1)/(D_2+1)]+1 )|...|( c_1j/T_1 )*( ln[(n+1)/(D_j+1)]+1 )|...|( c_1m/T_1 )*( ln[(n+1)/(D_m+1)]+1 )|
|__Doc_2__|( c_21/T_2 )*( ln[(n+1)/(D_1+1)]+1 )|( c_22/T_2 )*( ln[(n+1)/(D_2+1)]+1 )|...|( c_2j/T_2 )*( ln[(n+1)/(D_j+1)]+1 )|...|( c_2m/T_2 )*( ln[(n+1)/(D_m+1)]+1 )|
|__...__|...|...|...|...|...|...|
|__Doc_i__|( c_i1/T_i )*( ln[(n+1)/(D_1+1)]+1 )|( c_i2/T_i )*( ln[(n+1)/(D_2+1)]+1 )|...|( c_ij/T_i )*( ln[(n+1)/(D_j+1)]+1 )|...|( c_im/T_i )*( ln[(n+1)/(D_m+1)]+1 )|
|__...__|...|...|...|...|...|...|
|__Doc_n__|( c_n1/T_n )*( ln[(n+1)/(D_1+1)]+1 )|( c_n2/T_n )*( ln[(n+1)/(D_2+1)]+1 )|...|( c_nj/T_n )*( ln[(n+1)/(D_j+1)]+1 )|...|( c_nm/T_n )*( ln[(n+1)/(D_m+1)]+1 )|

#### then, normalize the weights so they are between 0 and 1 for each document
TFIDF_ij = W_ij / sqrt( sum( W_ij^2 ) )

TFIDF_11 = W_11 / sqrt(W_11 + W_12 + W_13 + W_14)

# Exercise 3
### Calculate (by hand) the TFIDF matrix for Doc_1 of the following count matrix:


|c |term_1|term_2|term_3|term_4|term_5|term_6|<font color='red'>T</font>|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|__Doc_1__|0|1|1|0|1|1|<font color='red'>4</font>|
|__Doc_2__|0|2|1|0|1|1|<font color='red'>5</font>|
|__Doc_3__|1|0|1|1|1|1|<font color='red'>5</font>|
|__<font color='blue'>D</font>__|<font color='blue'>1</font>|<font color='blue'>2</font>|<font color='blue'>3</font>|<font color='blue'>1</font>|<font color='blue'>3</font>|<font color='blue'>3</font>||

## 5.4 TFIDF Transformation Using scikit learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = ['This is the document.',
          'This document is the document.',
          'And this is the paper.']

t_vectorizer = TfidfVectorizer()
X = t_vectorizer.fit_transform(corpus)
words = t_vectorizer.get_feature_names()
tfidf_matrix = X.toarray()
print(words)
print(tfidf_matrix)

## 5.5 Topic Models

- A type of statistical model for discovering the abstract “topics” that occur in a collection of documents
- Useful for summarizing a large corpus that is too big to be read by a human
- Can think of it as a form of dimensionality reduction

### 5.5.1 Latent Dirichlet Allocation (LDA)
- A flavor of topic modeling; very popular since its inception in 2003 by David Blei
- Models documents as a collection of topics; models topics as a collection of words
- Assumptions
    - documents are made up of multiple topics
        - some topics are more present than others in a document
    - topics are the ranked words in the corpus
        - some words are more probable than others in a topic

![title](./lda.jpg)

### 5.5.2 Example LDA Code Using `scikit learn`

#### ABC News Headlines Corpus from Kaggle

- publish_date: Date of publishing for the article in yyyyMMdd format
- headline_text: Text of the headline in Ascii, English, lowercase
- Start Date: 2003-02-19 End Date: 2017-12-31
- Total Records: 1,103,663

Rohit Kulkarni (2017), A Million News Headlines [CSV Data file], doi:10.7910/DVN/SYBGZL, Retrieved from: https://www.kaggle.com/therohk/million-headlines/downloads/million-headlines.zip/8

In [ ]:
import pandas as pd
data = pd.read_csv('./abcnews-date-text.csv', error_bad_lines=False)
print(data.head())

corpus = list(data['headline_text'])
corpus = corpus[:10000]
print(corpus[:5])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# prepare the corpus
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
words = vectorizer.get_feature_names()

# initialize the LDA model with some parameters
n_topics = 30
lda = LatentDirichletAllocation(n_components=n_topics, learning_method='online', 
                                  random_state=0)

### Get the topic word weights

In [ ]:
topic_word_weights = lda.fit(X).components_
print('the shape of the LDA output is ' + str(topic_word_weights.shape))
print('there are ' + str(len(words)) + ' unique words in the corpus')
print('there are ' + str(n_topics) + ' topics in our model')

In [ ]:
# print the topic word weights
print(topic_word_weights)

### Get the document topic weights

In [ ]:
# the document topic weights
doc_topic_weights = lda.fit_transform(X)
print('there are ' + str(len(corpus)) + ' documents in the corpus')
print(doc_topic_weights.shape)
print(doc_topic_weights)

### Display the topics

In [ ]:
import numpy as np

def display_topics(model, vocab, n_top_words=10):
    """
    - argsort gives the indices of topic array to sort it
    - the slice 0:-11:-1 gets the last 10 elements of the indices array, which can
      then be used to access the top 10 occurring words in the topic
    """
    for topic_idx, topic in enumerate(model.components_):
        print('Topic ' + str(topic_idx) + ':')
        print('|'.join([vocab[i] for i in np.argsort(topic)[:-n_top_words-1:-1]]))
        
display_topics(lda, words)

#### Pros and Cons of LDA
- Pro: Simple to use, can get very far just tuning the number of topics
- Con: Evaluation is subjective and requires subject matter expertise

## 5.6 Other popular NLP packages

- Spacy
- Gensim

# Recap
- n grams counts how many times unique phrases of length n appear in a document
- n > 3 is rarely used
- While n grams are simple to calculate, the count matrix is sparse and requires a lot of memory to store => computationally inefficient 
- The goal of TFIDF is to de-bias the count matrix
- LDA is one flavor of many different topic modeling algorithms
- It can be used to summarize what a large, prohibitively long corpus of documents is about
- Evaluation is subjective and requires subject matter expertise 